In [1]:
!export AUTOGRAPH_VERBOSITY=0
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
import tensorflow as tf 
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
# tf.compat.v1.logging.set_verbosity(100)

In [2]:
%load_ext autoreload
%autoreload 2
import cantera as ct
import os
import gym 
import numpy as np 
import pandas as pd
from stable_baselines.common.policies import MlpPolicy, CnnLstmPolicy,MlpLstmPolicy
from stable_baselines.common.vec_env import SubprocVecEnv
from stable_baselines.bench import Monitor
from stable_baselines.results_plotter import load_results, ts2xy
from stable_baselines.common import set_global_seeds
from stable_baselines import ACKTR, PPO2
from envs.sim_env import SimEnv, NO_idx, CO_idx, O2_idx, H2O_idx
from envs.SimUtils import solvePhi_airSplit, equil, runMainBurner, correctNOx
import seaborn as sns

In [3]:
model = PPO2.load("Trained Models/PPO2_MlpLstmPolicy_9677c67_11202019_80000.zip")

In [4]:
from stable_baselines.common.vec_env import DummyVecEnv
sim_env = SimEnv()
env = DummyVecEnv([lambda: sim_env])
# env = SimEnv()
obs = env.reset()
env.action_space.high

array([17733.37587108,  1690.62375258,  9642.49689204])

In [5]:
model.set_env(env)

In [ ]:
game_over = False
results_list = []
while not game_over: 
    action, _states = model.predict(obs)
#     action = env.action_space.high[np.newaxis,:]
#     action = env.action_space.high[np.newaxis,:]
    obs, rewards, game_over, info = env.step(action)
    env.render(mode='human')
    g = env.envs[0].sec_stage_gas
    results_list.append([
        env.envs[0].age/1e-3, 
        g.T, 
        correctNOx(g.X[NO_idx], g.X[H2O_idx], g.X[O2_idx]),
        correctNOx(g.X[CO_idx], g.X[H2O_idx], g.X[O2_idx]),
        env.envs[0].mfc_main.mdot(0),
        env.envs[0].mfc_fuel_sec.mdot(0),
        env.envs[0].mfc_air_sec.mdot(0), 
        env.envs[0].remaining_main_burner_mass,
        env.envs[0].sec_fuel_remaining,
        env.envs[0].sec_air_remaining,
        rewards, ])

step|age_(ms)|T|phi_norm|NO|CO|Rem_Main|Rem_SecFuel|Rem_SecAir|Mdot_Main|Mdot_SecFuel|Mdot_SecAir|Max_Main|Max_SecFuel|Max_SecAir|Reward|Reward_Reactants|Reward_T|Reward_NO|Reward_CO
1| 0.00| 728.17| 0.95| 0.01| 0.25| 88.67| 1.69| 9.64| 1.65| 1.17| 0.00| 17733.38| 1690.62| 9642.50| 225.38|0.00| -373.38|498.76|100.00
2| 0.00| 863.60| 0.92| 0.02| 0.37| 88.67| 1.69| 9.64| 1.16| 0.00| 0.00| 17733.38| 1690.62| 9642.50| 293.83|0.00| -304.93|498.75|100.00
3| 0.00| 1012.24| 0.87| 0.03| 0.54| 88.67| 1.69| 9.64| 2.11| 0.00| 0.00| 17733.38| 1690.62| 9642.50| 385.69|0.00| -213.06|498.75|100.00
4| 0.00| 909.04| 0.88| 0.02| 0.45| 88.67| 1.69| 9.64| 1.89| 1.12| 1.13| 17733.38| 1690.62| 9642.50| 320.87|0.00| -277.88|498.75|100.00
5| 0.00| 837.41| 0.90| 0.02| 0.37| 88.67| 1.69| 9.64| 1.55| 1.45| 0.96| 17733.38| 1690.62| 9642.50| 279.03|0.00| -319.73|498.75|100.00
6| 0.01| 769.30| 0.92| 0.02| 0.30| 88.67| 1.69| 9.64| 0.00| 1.30| 0.00| 17733.38| 1690.62| 9642.50| 243.92|0.00| -354.83|498.76|100.00
7| 0.0

60| 0.06| 660.82| 0.95| 0.01| 0.19| 88.67| 1.69| 9.64| 2.26| 0.79| 0.00| 17733.38| 1690.62| 9642.50| 199.50|0.00| -399.26|498.76|100.00
61| 0.06| 660.81| 0.95| 0.01| 0.19| 88.67| 1.69| 9.64| 0.00| 0.00| 0.10| 17733.38| 1690.62| 9642.50| 199.50|0.00| -399.26|498.76|100.00
62| 0.06| 665.93| 0.94| 0.01| 0.20| 88.67| 1.69| 9.64| 1.47| 0.00| 0.00| 17733.38| 1690.62| 9642.50| 201.27|0.00| -397.49|498.76|100.00
63| 0.06| 664.98| 0.95| 0.01| 0.20| 88.67| 1.69| 9.64| 0.13| 0.40| 0.00| 17733.38| 1690.62| 9642.50| 200.94|0.00| -397.82|498.76|100.00
64| 0.06| 666.94| 0.94| 0.01| 0.20| 88.67| 1.69| 9.64| 0.91| 0.33| 0.00| 17733.38| 1690.62| 9642.50| 201.62|0.00| -397.13|498.76|100.00
65| 0.06| 674.20| 0.94| 0.01| 0.20| 88.67| 1.69| 9.64| 2.16| 0.00| 0.18| 17733.38| 1690.62| 9642.50| 204.20|0.00| -394.56|498.76|100.00
66| 0.07| 671.99| 0.94| 0.01| 0.20| 88.67| 1.69| 9.64| 0.72| 1.32| 0.35| 17733.38| 1690.62| 9642.50| 203.41|0.00| -395.35|498.76|100.00
67| 0.07| 675.37| 0.94| 0.01| 0.20| 88.67| 1.69|

123| 0.12| 653.72| 0.95| 0.01| 0.18| 88.67| 1.69| 9.64| 2.65| 1.81| 1.16| 17733.38| 1690.62| 9642.50| 197.09|0.00| -401.67|498.76|100.00
124| 0.12| 653.72| 0.95| 0.01| 0.18| 88.67| 1.69| 9.64| 0.00| 0.00| 0.00| 17733.38| 1690.62| 9642.50| 197.09|0.00| -401.67|498.76|100.00
125| 0.12| 652.34| 0.95| 0.01| 0.18| 88.67| 1.69| 9.64| 0.00| 0.83| 0.00| 17733.38| 1690.62| 9642.50| 196.63|0.00| -402.13|498.76|100.00
126| 0.13| 649.06| 0.95| 0.01| 0.17| 88.67| 1.69| 9.64| 0.00| 2.00| 0.00| 17733.38| 1690.62| 9642.50| 195.54|0.00| -403.22|498.76|100.00
127| 0.13| 647.32| 0.95| 0.01| 0.17| 88.67| 1.69| 9.64| 0.94| 2.09| 0.95| 17733.38| 1690.62| 9642.50| 194.97|0.00| -403.79|498.76|100.00
128| 0.13| 650.59| 0.95| 0.01| 0.17| 88.67| 1.69| 9.64| 2.52| 0.61| 0.00| 17733.38| 1690.62| 9642.50| 196.05|0.00| -402.71|498.76|100.00
129| 0.13| 651.41| 0.95| 0.01| 0.17| 88.67| 1.69| 9.64| 1.84| 1.40| 0.94| 17733.38| 1690.62| 9642.50| 196.32|0.00| -402.44|498.76|100.00
130| 0.13| 653.49| 0.95| 0.01| 0.18| 88.6

185| 0.18| 635.77| 0.95| 0.01| 0.16| 88.67| 1.69| 9.64| 0.61| 0.53| 1.09| 17733.38| 1690.62| 9642.50| 191.26|0.00| -407.50|498.76|100.00
186| 0.19| 636.92| 0.95| 0.01| 0.16| 88.67| 1.69| 9.64| 2.54| 1.68| 0.36| 17733.38| 1690.62| 9642.50| 191.62|0.00| -407.14|498.76|100.00
187| 0.19| 638.60| 0.95| 0.01| 0.16| 88.67| 1.69| 9.64| 1.56| 0.01| 0.00| 17733.38| 1690.62| 9642.50| 192.15|0.00| -406.60|498.76|100.00
188| 0.19| 638.92| 0.95| 0.01| 0.16| 88.67| 1.69| 9.64| 0.60| 0.35| 0.84| 17733.38| 1690.62| 9642.50| 192.25|0.00| -406.50|498.76|100.00
189| 0.19| 637.79| 0.95| 0.01| 0.16| 88.67| 1.69| 9.64| 0.84| 2.09| 0.00| 17733.38| 1690.62| 9642.50| 191.89|0.00| -406.86|498.76|100.00
190| 0.19| 638.54| 0.95| 0.01| 0.16| 88.67| 1.69| 9.64| 0.70| 0.00| 0.00| 17733.38| 1690.62| 9642.50| 192.13|0.00| -406.62|498.76|100.00
191| 0.19| 639.09| 0.95| 0.01| 0.16| 88.67| 1.69| 9.64| 1.27| 0.85| 0.70| 17733.38| 1690.62| 9642.50| 192.31|0.00| -406.45|498.76|100.00
192| 0.19| 639.49| 0.95| 0.01| 0.16| 88.6

247| 0.25| 640.33| 0.95| 0.01| 0.15| 88.67| 1.69| 9.64| 0.06| 1.29| 0.00| 17733.38| 1690.62| 9642.50| 192.69|0.00| -406.05|498.76|100.00
248| 0.25| 640.33| 0.95| 0.01| 0.15| 88.67| 1.69| 9.64| 0.00| 0.00| 0.00| 17733.38| 1690.62| 9642.50| 192.69|0.00| -406.05|498.76|100.00
249| 0.25| 640.99| 0.95| 0.01| 0.15| 88.67| 1.69| 9.64| 0.87| 0.10| 0.96| 17733.38| 1690.62| 9642.50| 192.90|0.00| -405.84|498.76|100.00
250| 0.25| 640.24| 0.95| 0.01| 0.15| 88.67| 1.69| 9.64| 0.15| 1.16| 0.33| 17733.38| 1690.62| 9642.50| 192.66|0.00| -406.08|498.76|100.00
251| 0.25| 639.73| 0.95| 0.01| 0.15| 88.67| 1.69| 9.64| 1.27| 2.07| 0.02| 17733.38| 1690.62| 9642.50| 192.50|0.00| -406.24|498.76|100.00
252| 0.25| 639.33| 0.95| 0.01| 0.15| 88.67| 1.69| 9.64| 0.18| 0.73| 0.00| 17733.38| 1690.62| 9642.50| 192.37|0.00| -406.37|498.76|100.00
253| 0.25| 639.00| 0.95| 0.01| 0.15| 88.67| 1.69| 9.64| 2.31| 3.00| 0.00| 17733.38| 1690.62| 9642.50| 192.27|0.00| -406.47|498.76|100.00
254| 0.25| 639.81| 0.95| 0.01| 0.15| 88.6

...

6178| 6.18| 646.02| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 0.59| 0.45| 0.24| 17733.38| 1690.62| 9642.50| -41.23|0.03| -404.21|498.76|100.00
6179| 6.18| 646.05| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 1.13| 0.10| 1.13| 17733.38| 1690.62| 9642.50| -41.33|0.03| -404.20|498.76|100.00
6180| 6.18| 646.04| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 1.02| 1.55| 1.09| 17733.38| 1690.62| 9642.50| -41.45|0.03| -404.21|498.76|100.00
6181| 6.18| 646.04| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 0.00| 0.00| 1.08| 17733.38| 1690.62| 9642.50| -41.56|0.03| -404.21|498.76|100.00
6182| 6.18| 646.01| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 0.00| 0.83| 0.15| 17733.38| 1690.62| 9642.50| -41.69|0.03| -404.22|498.76|100.00
6183| 6.18| 646.00| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 0.00| 0.52| 0.78| 17733.38| 1690.62| 9642.50| -41.81|0.03| -404.22|498.76|100.00
6184| 6.18| 645.95| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 0.20| 1.71| 0.00| 17733.38| 1690.62| 9642.50| -41.94|0.03| -404.24|498.76|100.00
6185| 6.19| 645.92| 0.95| 0.01| 0.

6252| 6.25| 646.04| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 3.03| 2.64| 0.00| 17733.38| 1690.62| 9642.50| -49.80|0.03| -404.21|498.76|100.00
6253| 6.25| 645.98| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 0.10| 2.01| 0.00| 17733.38| 1690.62| 9642.50| -49.93|0.03| -404.23|498.76|100.00
6254| 6.25| 646.02| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 1.22| 0.00| 0.29| 17733.38| 1690.62| 9642.50| -50.04|0.03| -404.21|498.76|100.00
6255| 6.26| 645.97| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 0.72| 2.23| 0.00| 17733.38| 1690.62| 9642.50| -50.17|0.03| -404.23|498.76|100.00
6256| 6.26| 645.96| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 0.00| 0.29| 0.00| 17733.38| 1690.62| 9642.50| -50.29|0.03| -404.23|498.76|100.00
6257| 6.26| 645.95| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 0.00| 0.37| 0.37| 17733.38| 1690.62| 9642.50| -50.41|0.03| -404.24|498.76|100.00
6258| 6.26| 645.94| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 1.95| 2.43| 0.41| 17733.38| 1690.62| 9642.50| -50.53|0.03| -404.24|498.76|100.00
6259| 6.26| 645.99| 0.95| 0.01| 0.

6325| 6.33| 646.52| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 2.07| 2.75| 0.00| 17733.38| 1690.62| 9642.50| -58.30|0.03| -404.05|498.76|100.00
6326| 6.33| 646.52| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 2.26| 2.34| 1.69| 17733.38| 1690.62| 9642.50| -58.42|0.03| -404.05|498.76|100.00
6327| 6.33| 646.55| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 1.24| 0.52| 0.00| 17733.38| 1690.62| 9642.50| -58.53|0.03| -404.04|498.76|100.00
6328| 6.33| 646.57| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 0.82| 0.00| 0.00| 17733.38| 1690.62| 9642.50| -58.64|0.03| -404.03|498.76|100.00
6329| 6.33| 646.60| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 1.32| 0.45| 0.00| 17733.38| 1690.62| 9642.50| -58.75|0.03| -404.02|498.76|100.00
6330| 6.33| 646.55| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 0.00| 1.82| 0.00| 17733.38| 1690.62| 9642.50| -58.89|0.03| -404.04|498.76|100.00
6331| 6.33| 646.57| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 1.54| 0.99| 1.34| 17733.38| 1690.62| 9642.50| -59.00|0.03| -404.04|498.76|100.00
6332| 6.33| 646.54| 0.95| 0.01| 0.

6402| 6.40| 646.84| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 1.05| 1.48| 0.00| 17733.38| 1690.62| 9642.50| -67.55|0.03| -403.95|498.76|100.00
6403| 6.40| 646.85| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 0.40| 0.06| 0.02| 17733.38| 1690.62| 9642.50| -67.67|0.03| -403.94|498.76|100.00
6404| 6.40| 646.87| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 0.84| 0.00| 1.99| 17733.38| 1690.62| 9642.50| -67.78|0.03| -403.93|498.76|100.00
6405| 6.41| 646.92| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 1.52| 0.00| 0.00| 17733.38| 1690.62| 9642.50| -67.89|0.03| -403.92|498.76|100.00
6406| 6.41| 646.92| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 0.73| 0.75| 1.12| 17733.38| 1690.62| 9642.50| -68.01|0.03| -403.92|498.76|100.00
6407| 6.41| 646.95| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 2.41| 1.79| 0.69| 17733.38| 1690.62| 9642.50| -68.13|0.03| -403.91|498.76|100.00
6408| 6.41| 647.01| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 2.38| 0.45| 0.00| 17733.38| 1690.62| 9642.50| -68.23|0.03| -403.89|498.76|100.00
6409| 6.41| 647.02| 0.95| 0.01| 0.

6476| 6.48| 646.94| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 3.36| 0.00| 0.00| 17733.38| 1690.62| 9642.50| -76.72|0.03| -403.91|498.76|100.00
6477| 6.48| 646.90| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 0.07| 1.66| 0.17| 17733.38| 1690.62| 9642.50| -76.86|0.03| -403.93|498.76|100.00
6478| 6.48| 646.86| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 0.88| 2.22| 0.00| 17733.38| 1690.62| 9642.50| -77.00|0.03| -403.94|498.76|100.00
6479| 6.48| 646.83| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 0.77| 1.56| 0.00| 17733.38| 1690.62| 9642.50| -77.13|0.03| -403.95|498.76|100.00
6480| 6.48| 646.79| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 0.00| 1.48| 0.00| 17733.38| 1690.62| 9642.50| -77.27|0.03| -403.96|498.76|100.00
6481| 6.48| 646.76| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 0.00| 0.87| 0.66| 17733.38| 1690.62| 9642.50| -77.41|0.03| -403.97|498.76|100.00
6482| 6.48| 646.73| 0.95| 0.01| 0.03| 88.66| 1.68| 9.64| 0.00| 1.03| 0.00| 17733.38| 1690.62| 9642.50| -77.54|0.03| -403.98|498.76|100.00
6483| 6.48| 646.66| 0.95| 0.01| 0.

In [ ]:
import pdb 
from tqdm import tqdm_notebook
env.reset()
for i in tqdm_notebook(range(0, 16000)):
    action, _states = model.predict(obs)
#     action = env.action_space.high[np.newaxis,:]
#     action = env.action_space.high[np.newaxis,:]
    obs, rewards, game_over, info = env.step(action)
    if game_over:
        pdb.set_trace()
    if i == 15999:
        pdb.set_trace()

In [ ]:
env.envs[0].reward_T, env.envs[0].reward_NO, env.envs[0].reward_reactants

In [ ]:
df = pd.DataFrame(data=np.vstack(results_list), columns=['age', 'T', 'NO', 'CO', 'mdot_main', 'mdot_fuel_sec', 'mdot_air_sec', 'rem_main', 'rem_fuel_sec', 'rem_air_sec', 'reward'])
df = df.iloc[0:-1]
# df.to_csv("simResults_500000.csv", index=False)

In [ ]:
df

In [ ]:
df['mdot_main'].hist()

In [ ]:
df.plot('age', 'reward')

In [ ]:
env.envs[0].action_space.high[2]